In [2]:
import pandas as pd
import json
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.sentiment import SentimentIntensityAnalyzer

# Download required NLTK resources
nltk.download("vader_lexicon")

# Initialize Sentiment Analyzer
sia = SentimentIntensityAnalyzer()

# Load Reddit data (Ensure this file exists)
try:
    with open("reddit_data_location.json", "r") as f:
        reddit_data = json.load(f)
except FileNotFoundError:
    print("❌ Error: 'reddit_data_location.json' not found! Ensure the file exists in the working directory.")
    exit()

df = pd.DataFrame(reddit_data)

# Ensure 'content' column exists
if "content" not in df.columns:
    print("❌ Error: 'content' column missing in dataset!")
    exit()

# Function to classify sentiment using VADER
def classify_sentiment(text):
    if not isinstance(text, str) or text.strip() == "":
        return "Neutral"
    sentiment_score = sia.polarity_scores(text)["compound"]
    if sentiment_score >= 0.05:
        return "Positive"
    elif sentiment_score <= -0.05:
        return "Negative"
    else:
        return "Neutral"

# Crisis risk classification based on keyword matching
high_risk_terms = [
    "suicide", "end my life", "kill myself", "no reason to live", "overdose", 
    "self-harm", "cutting", "hanging", "jump off", "take my life"
]
moderate_risk_terms = [
    "depressed", "hopeless", "struggling", "feeling lost", "need help", 
    "seeking therapy", "panic attack", "overwhelmed", "burnout", "lonely"
]

def classify_risk(text):
    if not isinstance(text, str):
        return "Low Concern"
    
    text_lower = text.lower()

    # Check for high-risk terms
    if any(term in text_lower for term in high_risk_terms):
        return "High-Risk"
    
    # Check for moderate-risk terms
    if any(term in text_lower for term in moderate_risk_terms):
        return "Moderate Concern"

    return "Low Concern"

# Apply sentiment and risk classification
df["sentiment"] = df["content"].apply(classify_sentiment)
df["risk_level"] = df["content"].apply(classify_risk)

# Save classified data
df.to_csv("reddit_data_classified.csv", index=False)

# ✅ Ensure plots work without VS Code renderer
plt.switch_backend("Agg")  # Use non-interactive backend

# Plot sentiment distribution
plt.figure(figsize=(8, 5))
sns.countplot(data=df, x="sentiment", palette="coolwarm")
plt.title("Distribution of Sentiments in Reddit Posts")
plt.xlabel("Sentiment")
plt.ylabel("Count")
plt.savefig("sentiment_distribution.png")  # Save plot instead of displaying
plt.close()

# Plot risk level distribution
plt.figure(figsize=(8, 5))
sns.countplot(data=df, x="risk_level", palette="Reds")
plt.title("Distribution of Crisis Risk Levels")
plt.xlabel("Risk Level")
plt.ylabel("Count")
plt.savefig("risk_level_distribution.png")  # Save plot instead of displaying
plt.close()

print("\n🎉 Sentiment and Risk Classification Complete! Data saved to 'reddit_data_classified.csv'.")
print("📊 Plots saved as 'sentiment_distribution.png' and 'risk_level_distribution.png'.")


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!



🎉 Sentiment and Risk Classification Complete! Data saved to 'reddit_data_classified.csv'.
📊 Plots saved as 'sentiment_distribution.png' and 'risk_level_distribution.png'.


/tmp/ipykernel_6168/4043076282.py:79: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(data=df, x="sentiment", palette="coolwarm")
/tmp/ipykernel_6168/4043076282.py:88: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(data=df, x="risk_level", palette="Reds")
